In [1]:
import random
import numpy as np
import torch
import os
import pathlib
import pickle
import pandas as pd
torch.manual_seed(42)
torch.cuda.manual_seed(42)
from Performance import performance_evaluation
perfs = performance_evaluation()

from Performance import H2h_Regret_Evaluation
h2h_regret_eva = H2h_Regret_Evaluation()

from Data import Data_Simulator
DS_Obj = Data_Simulator()

from Oracle import Oracle_Processing
Oracle_Proc = Oracle_Processing()

from OLS import OLS_Processing
OLS_Proc = OLS_Processing()

from DDR import DDR_Processing
DDR_Proc = DDR_Processing()

from PYEPO import EPO_Processing
PYEPO_Proc = EPO_Processing()

from Data_Load_Store import Load_Store_Methods
Data_LSM = Load_Store_Methods()

In [2]:
def run_all_approaches(DataPath,mu_all,lamb_all,arcs, grid,mis,bump,iteration_all,num_feat,data_generation_process):
        
        x_test_all,c_test_all,x_train_all,c_train_all,noise_train_all,noise_test_all,W_star_all = Data_LSM.load_input_data(DataPath)

        cost_Oracle_Post_all,cost_Oracle_Ante_all = Oracle_Proc.Implement_Oracle(arcs, grid,mis,bump,\
                                                                    W_star_all,x_test_all,noise_test_all,\
                                                                    iteration_all,num_feat,data_generation_process)
        
        cost_OLS_Post_all,cost_OLS_Ante_all = OLS_Proc.Implement_OLS(arcs, grid,mis,bump,\
                                                            W_star_all,x_test_all,noise_test_all,x_train_all,c_train_all,\
                                                            iteration_all,num_feat,data_generation_process)

        cost_DDR_Post_all,cost_DDR_Ante_all = DDR_Proc.Implement_DDR(mu_all,lamb_all,arcs, grid,mis,bump,\
                                                                    W_star_all,x_test_all,noise_test_all,x_train_all,c_train_all,\
                                                                        iteration_all,num_feat,data_generation_process)

        Data_LSM.store_Oracle_OLS_DDR_Cost(DataPath,cost_Oracle_Post_all,cost_Oracle_Ante_all,\
                                  cost_OLS_Post_all,cost_OLS_Ante_all,\
                                  cost_DDR_Post_all,cost_DDR_Ante_all)

In [3]:
def run_EPO_approaches(DataPath,method_names,arcs, grid,mis,bump,iteration_all,num_feat,data_generation_process):
        
        x_test_all,c_test_all,x_train_all,c_train_all,noise_train_all,noise_test_all,W_star_all = Data_LSM.load_input_data(DataPath)

        batch_size = 20
        num_epochs = 1000
        # from PYEPO import PyEPO_Method
        # epo_runner = PyEPO_Method()
        method_names = ["spo+"]
        cost_EPO_Post_all,cost_EPO_Ante_all = PYEPO_Proc.Implement_EPO(DataPath,iteration_all,batch_size,num_epochs,method_names,\
                                                    W_star_all,bump,x_train_all,c_train_all,x_test_all,noise_test_all,\
                                                    arcs,grid,perfs,num_feat,mis,data_generation_process)

        with open(DataPath+'cost_'+method_names[0]+'_Post_all.pkl', "wb") as tf:
            pickle.dump(cost_EPO_Post_all,tf)
        with open(DataPath+'cost_'+method_names[0]+'_Ante_all.pkl', "wb") as tf:
            pickle.dump(cost_EPO_Ante_all,tf)

### Parameters

In [4]:
grid = (3,3) # grid size
from Network import network_design
Network = network_design()
arcs,arc_index_mapping = Network._getArcs(grid)

num_test = 1000
lower = 0 # coef lower bound
upper = 1 # coef upper bound
d = (grid[0] - 1) * (grid[0] - 1) * 2 + 2 * (grid[0] - 1) # num of arcs
num_nodes = grid[0]*grid[0]
coef_seed = 1

x_dist = 'uniform'
e_dist = 'normal'
x_low = -2
x_up = 2
x_mean = 2
x_var = 2
bump = 100

In [5]:
data_generation_process = "SPO_Data_Generation"
# data_generation_process = "DDR_Data_Generation"
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
project_directory = os.path.dirname(os.path.dirname(os.path.dirname(parent_directory)))
DataPath_Parent = project_directory + '/Data_JOC_R1/Shortest_Path/Various_Settings_' + data_generation_process + f'_coef_seed={coef_seed}/{grid[0]}by{grid[1]}_grid/'
pathlib.Path(DataPath_Parent).mkdir(parents=True, exist_ok=True)
print("DataPath_parent:", DataPath_Parent)

DataPath_parent: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data_JOC_R1/Shortest_Path/Various_Settings_SPO_Data_Generation_coef_seed=1/3by3_grid/


#### Impact of sample size

In [ ]:
num_feat = 5 # size of feature
p = num_feat
deg = 1.0 # polynomial degree
mis = deg # model misspecification
e = 0.5 # scale of normal std or the range of uniform. For the error term
alpha = e # scale of normal std or the range of uniform. For the error term
iteration_all = np.arange(0,100)
batch_size = 20
num_epochs = 1000
num_train_all = [50,100,200,500,1000]
mu_all = np.asarray([0.4,0.5,0.6])
lamb_all = np.asarray([0.4,0.5,0.6])
for num_train in num_train_all:
    DataPath = DataPath_Parent + f"data_size={num_train}_deg={deg}_e={e}_p={p}_num_test={num_test}"+"_x_dist="+x_dist+"_e_dist="+e_dist+"/"
    pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)
    print("DataPath:", DataPath)
    
    x_test_all, c_test_all, x_train_all, c_train_all,noise_train_all,noise_test_all,W_star_all = \
    DS_Obj.Simulator(DataPath,lower, upper, p, d, coef_seed,iteration_all,num_test, num_train, alpha,mis,data_generation_process,x_dist, e_dist, x_low, x_up, x_mean, x_var, bump)

    Data_LSM.store_input_data(DataPath,x_test_all,c_test_all,x_train_all,c_train_all,noise_test_all,noise_train_all,W_star_all)

DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data_JOC_R1/Shortest_Path/Various_Settings_SPO_Data_Generation_coef_seed=1/3by3_grid/data_size=100_deg=1.0_e=0.5_p=5_num_test=1000_x_dist=uniform_e_dist=normal/


In [7]:
for num_train in num_train_all:
    DataPath = DataPath_Parent + f"data_size={num_train}_deg={deg}_e={e}_p={p}_num_test={num_test}"+"_x_dist="+x_dist+"_e_dist="+e_dist+"/"
    print("DataPath:", DataPath)
    run_all_approaches(DataPath,mu_all,lamb_all,arcs, grid,mis,bump,iteration_all,num_feat,data_generation_process)

DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data_JOC_R1/Shortest_Path/Various_Settings_SPO_Data_Generation_coef_seed=1/3by3_grid/data_size=100_deg=1.0_e=0.5_p=5_num_test=1000_x_dist=uniform_e_dist=normal/
Set parameter Username
Academic license - for non-commercial use only - expires 2026-03-13
Oracle: iter= 20 ,cost_Oracle_Ante= 15.172642031123184
Oracle: iter= 40 ,cost_Oracle_Ante= 14.600967169194192
Oracle: iter= 60 ,cost_Oracle_Ante= 15.016002191297218
Oracle: iter= 80 ,cost_Oracle_Ante= 14.89354010044007
OLS: iter= 20 ,cost_OLS_Ante= 15.342978625531082
OLS: iter= 40 ,cost_OLS_Ante= 14.653128870271075
OLS: iter= 60 ,cost_OLS_Ante= 15.116680536722342
OLS: iter= 80 ,cost_OLS_Ante= 15.042281568159728
DDR: iter= 20 ,mu= 0.6 ,lamb= 0.6 ,cost_DDR_Ante = 15.337401682302461
DDR: iter= 40 ,mu= 0.6 ,lamb= 0.6 ,cost_DDR_Ante = 14.653867819231504
DDR: iter= 60 ,mu= 0.6 ,lamb= 0.6 ,cost_DDR_Ante = 15.118742798669407
DDR: iter= 80 ,mu= 0.6 ,lamb= 0.6 ,cost_DDR_Ante 

In [ ]:
# for num_train in num_train_all:
#     DataPath = DataPath_Parent + f"data_size={num_train}_deg={deg}_e={e}_p={p}_num_test={num_test}"+"_x_dist="+x_dist+"_e_dist="+e_dist+"/"
#     print("DataPath:", DataPath)
#     method_names = ["spo+"]
#     run_EPO_approaches(DataPath,method_names,arcs, grid,mis,bump,iteration_all,num_feat,data_generation_process)

In [8]:
regret_N_ante_Dict = {}; h2h_N_ante_Dict = {}
for mu in mu_all:
    for lamb in lamb_all:
        regret_N_ante = np.zeros(len(num_train_all)); h2h_N_ante = np.zeros(len(num_train_all))
        _index = 0
        for num_train in num_train_all:
            DataPath = DataPath_Parent + f"data_size={num_train}_deg={deg}_e={e}_p={p}_num_test={num_test}"+"_x_dist="+x_dist+"_e_dist="+e_dist+"/"
            print(DataPath)
            
            cost_Oracle_Post_all,cost_Oracle_Ante_all,cost_OLS_Post_all,cost_OLS_Ante_all,cost_DDR_Post_all,cost_DDR_Ante_all = Data_LSM.load_cost_data(DataPath)
            # with open(DataPath+'cost_spo+_Ante_all.pkl', "rb") as tf:
            #     cost_SPO_Ante_all = pickle.load(tf)
            h2h_N_ante[_index], regret_N_ante[_index] = h2h_regret_eva.calculate_h2h_regret(mu,lamb,iteration_all,\
                                cost_DDR_Post_all,cost_OLS_Post_all,cost_Oracle_Post_all,\
                                    cost_DDR_Ante_all,cost_OLS_Ante_all,cost_Oracle_Ante_all)
            _index = _index + 1
        regret_N_ante_Dict[mu,lamb] = regret_N_ante
        h2h_N_ante_Dict[mu,lamb] = h2h_N_ante

/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data_JOC_R1/Shortest_Path/Various_Settings_SPO_Data_Generation_coef_seed=1/3by3_grid/data_size=100_deg=1.0_e=0.5_p=5_num_test=1000_x_dist=uniform_e_dist=normal/
/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data_JOC_R1/Shortest_Path/Various_Settings_SPO_Data_Generation_coef_seed=1/3by3_grid/data_size=100_deg=1.0_e=0.5_p=5_num_test=1000_x_dist=uniform_e_dist=normal/
/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data_JOC_R1/Shortest_Path/Various_Settings_SPO_Data_Generation_coef_seed=1/3by3_grid/data_size=100_deg=1.0_e=0.5_p=5_num_test=1000_x_dist=uniform_e_dist=normal/
/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data_JOC_R1/Shortest_Path/Various_Settings_SPO_Data_Generation_coef_seed=1/3by3_grid/data_size=100_deg=1.0_e=0.5_p=5_num_test=1000_x_dist=uniform_e_dist=normal/
/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data_JOC_R1/Shortest_Path/Various_Se

#### Impact of number of feature

In [ ]:
deg = 1.0 # polynomial degree
mis = deg # model misspecification
e = 0.5 # scale of normal std or the range of uniform. For the error term
alpha = e # scale of normal std or the range of uniform. For the error term
num_train = 100
num_feat_all = [1,3,5,7,10,15]
for num_feat in num_feat_all:
    p = num_feat
    if p != 5:
        DataPath = DataPath_Parent + f"data_size={num_train}_deg={deg}_e={e}_p={p}_num_test={num_test}"+"_x_dist="+x_dist+"_e_dist="+e_dist+"/"
        pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)
        print("DataPath:", DataPath)

        x_test_all, c_test_all, x_train_all, c_train_all,noise_train_all,noise_test_all,W_star_all \
        = DS_Obj.Simulator(DataPath,lower, upper, p, d, coef_seed,iteration_all,num_test, num_train, alpha,mis,data_generation_process,x_dist, e_dist, x_low, x_up, x_mean, x_var, bump)

        Data_LSM.store_input_data(DataPath,x_test_all,c_test_all,x_train_all,c_train_all,noise_test_all,noise_train_all,W_star_all)

In [ ]:
for num_feat in num_feat_all:
    p = num_feat
    if p != 5:
        DataPath = DataPath_Parent + f"data_size={num_train}_deg={deg}_e={e}_p={p}_num_test={num_test}"+"_x_dist="+x_dist+"_e_dist="+e_dist+"/"
        pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)
        print("DataPath:", DataPath)        
        run_all_approaches(DataPath,mu_all,lamb_all,arcs, grid,mis,bump,iteration_all,num_feat,data_generation_process)

In [ ]:
for num_feat in num_feat_all:
    p = num_feat
    if p != 5:
        DataPath = DataPath_Parent + f"data_size={num_train}_deg={deg}_e={e}_p={p}_num_test={num_test}"+"_x_dist="+x_dist+"_e_dist="+e_dist+"/"
        pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)
        print("DataPath:", DataPath)    
        run_EPO_approaches(DataPath,["spo+"],arcs, grid,mis,bump,iteration_all,num_feat,data_generation_process)

In [ ]:
regret_P_ante = np.zeros(len(num_feat_all)); h2h_P_ante = np.zeros(len(num_feat_all))
mu = mu_all[0]
lamb = lamb_all[0]
_index = 0
for num_feat in num_feat_all:
    p = num_feat
    DataPath = DataPath_Parent + f"data_size={num_train}_deg={deg}_e={e}_p={p}_num_test={num_test}"+"_x_dist="+x_dist+"_e_dist="+e_dist+"/"
    print("DataPath:", DataPath)        

    cost_Oracle_Post_all,cost_Oracle_Ante_all,cost_OLS_Post_all,cost_OLS_Ante_all,cost_DDR_Post_all,cost_DDR_Ante_all = Data_LSM.load_cost_data(DataPath)
    
    h2h_P_ante[_index], regret_P_ante[_index] = h2h_regret_eva.calculate_h2h_regret(mu,lamb,iteration_all,\
                         cost_DDR_Post_all,cost_OLS_Post_all,cost_Oracle_Post_all,\
                            cost_DDR_Ante_all,cost_OLS_Ante_all,cost_Oracle_Ante_all)
    _index = _index + 1

#### Impact of alpha

In [ ]:
deg = 1.0 # polynomial degree
mis = deg # model misspecification
num_train = 100
num_feat = 5
p = num_feat
e_all = [0.25,0.5,0.75,1.0]
for e in e_all:
    alpha = e # scale of normal std or the range of uniform. For the error term
    if e != 0.5:
        DataPath = DataPath_Parent + f"data_size={num_train}_deg={deg}_e={e}_p={p}_num_test={num_test}"+"_x_dist="+x_dist+"_e_dist="+e_dist+"/"
        pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)
        print("DataPath:", DataPath)
        x_test_all, c_test_all, x_train_all, c_train_all,noise_train_all,noise_test_all,W_star_all \
        = DS_Obj.Simulator(DataPath,lower, upper, p, d, coef_seed,iteration_all,num_test, num_train, alpha,mis,data_generation_process,x_dist, e_dist, x_low, x_up, x_mean, x_var, bump)

        Data_LSM.store_input_data(DataPath,x_test_all,c_test_all,x_train_all,c_train_all,noise_test_all,noise_train_all,W_star_all)

In [ ]:
for e in e_all:
    alpha = e 
    if e != 0.5:
        DataPath = DataPath_Parent + f"data_size={num_train}_deg={deg}_e={e}_p={p}_num_test={num_test}"+"_x_dist="+x_dist+"_e_dist="+e_dist+"/"
        print("DataPath:", DataPath)
        run_all_approaches(DataPath,mu_all,lamb_all,arcs, grid,mis,bump,iteration_all,num_feat,data_generation_process)

In [ ]:
for e in e_all:
    alpha = e 
    if e != 0.5:
        DataPath = DataPath_Parent + f"data_size={num_train}_deg={deg}_e={e}_p={p}_num_test={num_test}"+"_x_dist="+x_dist+"_e_dist="+e_dist+"/"
        print("DataPath:", DataPath)
        run_EPO_approaches(DataPath,["spo+"],arcs, grid,mis,bump,iteration_all,num_feat,data_generation_process)

In [ ]:
regret_e_ante = np.zeros(len(e_all)); h2h_e_ante = np.zeros(len(e_all))
mu = mu_all[0]
lamb = lamb_all[0]
_index = 0
for e in e_all:
    alpha = e 
    DataPath = DataPath_Parent + f"data_size={num_train}_deg={deg}_e={e}_p={p}_num_test={num_test}"+"_x_dist="+x_dist+"_e_dist="+e_dist+"/"
    print("DataPath:", DataPath)

    cost_Oracle_Post_all,cost_Oracle_Ante_all,cost_OLS_Post_all,cost_OLS_Ante_all,cost_DDR_Post_all,cost_DDR_Ante_all = Data_LSM.load_cost_data(DataPath)
    
    h2h_e_ante[_index], regret_e_ante[_index] = h2h_regret_eva.calculate_h2h_regret(mu,lamb,iteration_all,\
                         cost_DDR_Post_all,cost_OLS_Post_all,cost_Oracle_Post_all,\
                            cost_DDR_Ante_all,cost_OLS_Ante_all,cost_Oracle_Ante_all)
    _index = _index + 1

#### Impact of model misspecification

In [ ]:
e = 0.5
alpha = e
num_train = 100
num_feat = 5
p = num_feat
deg_all = [0.4,0.6,0.8,1.0,1.2,1.4,1.6,2.0,3.0,4.0,8.0]
for deg in deg_all:
    mis = deg # model misspecification
    if deg != 1.0:
        DataPath = DataPath_Parent + f"data_size={num_train}_deg={deg}_e={e}_p={p}_num_test={num_test}"+"_x_dist="+x_dist+"_e_dist="+e_dist+"/"
        pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)
        print("DataPath:", DataPath)

        x_test_all, c_test_all, x_train_all, c_train_all,noise_train_all,noise_test_all,W_star_all \
        = DS_Obj.Simulator(DataPath,lower, upper, p, d, coef_seed,iteration_all,num_test, num_train, alpha,mis,data_generation_process,x_dist, e_dist, x_low, x_up, x_mean, x_var, bump)

        Data_LSM.store_input_data(DataPath,x_test_all,c_test_all,x_train_all,c_train_all,noise_test_all,noise_train_all,W_star_all)

In [ ]:
for deg in deg_all:
    mis = deg # model misspecification
    if deg != 1.0:
        DataPath = DataPath_Parent + f"data_size={num_train}_deg={deg}_e={e}_p={p}_num_test={num_test}"+"_x_dist="+x_dist+"_e_dist="+e_dist+"/"
        print("DataPath:", DataPath)
        run_all_approaches(DataPath,mu_all,lamb_all,arcs, grid,mis,bump,iteration_all,num_feat,data_generation_process)

In [ ]:
for deg in deg_all:
    mis = deg # model misspecification
    if deg != 1.0:
        DataPath = DataPath_Parent + f"data_size={num_train}_deg={deg}_e={e}_p={p}_num_test={num_test}"+"_x_dist="+x_dist+"_e_dist="+e_dist+"/"
        print("DataPath:", DataPath)
        run_EPO_approaches(DataPath,["spo+"],arcs, grid,mis,bump,iteration_all,num_feat,data_generation_process)

In [ ]:
regret_d_ante = np.zeros(len(deg_all)); h2h_d_ante = np.zeros(len(deg_all))
mu = mu_all[0]
lamb = lamb_all[0]
_index = 0
for deg in deg_all:
    mis = deg # model misspecification
    DataPath = DataPath_Parent + f"data_size={num_train}_deg={deg}_e={e}_p={p}_num_test={num_test}"+"_x_dist="+x_dist+"_e_dist="+e_dist+"/"
    print("DataPath:", DataPath)

    cost_Oracle_Post_all,cost_Oracle_Ante_all,cost_OLS_Post_all,cost_OLS_Ante_all,cost_DDR_Post_all,cost_DDR_Ante_all = Data_LSM.load_cost_data(DataPath)
    
    h2h_d_ante[_index], regret_d_ante[_index] = h2h_regret_eva.calculate_h2h_regret(mu,lamb,iteration_all,\
                         cost_DDR_Post_all,cost_OLS_Post_all,cost_Oracle_Post_all,\
                            cost_DDR_Ante_all,cost_OLS_Ante_all,cost_Oracle_Ante_all)
    _index = _index + 1

## Figures

### Plot Figure

In [ ]:
Result_dir = DataPath_Parent + "Result/"
pathlib.Path(Result_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize = (5, 5))
p_indices = [0,1,3,4,5]
plt.plot(h2h_P_ante[p_indices], regret_P_ante[p_indices], color='green', marker = "*", label = 'p', linestyle = 'None')
plt.text(h2h_P_ante[p_indices[0]] + 0.01, regret_P_ante[p_indices[0]] - 0.005, 'p='+str(num_feat_all[p_indices[0]]), color='green')
plt.text(h2h_P_ante[p_indices[1]] + 0.01, regret_P_ante[p_indices[1]] - 0.005, 'p='+str(num_feat_all[p_indices[1]]), color='green')
plt.text(h2h_P_ante[p_indices[2]] - 0.05, regret_P_ante[p_indices[2]] - 0.005, 'p='+str(num_feat_all[p_indices[2]]), color='green')
plt.text(h2h_P_ante[p_indices[3]] - 0.05, regret_P_ante[p_indices[3]] - 0.005, 'p='+str(num_feat_all[p_indices[3]]), color='green')
plt.text(h2h_P_ante[p_indices[4]] - 0.065, regret_P_ante[p_indices[4]] - 0.005, 'p='+str(num_feat_all[p_indices[4]]), color='green')

plt.plot(h2h_d_ante[0], regret_d_ante[0], color='#003D7C', marker = "o", label = 'd', linestyle = 'None')
# plt.plot(h2h_d[1], mci_d[1], color='#003D7C', marker = "o", linestyle = 'None')
plt.plot(h2h_d_ante[1], regret_d_ante[1], color='#003D7C', marker = "o", linestyle = 'None')
plt.plot(h2h_d_ante[2], regret_d_ante[2], color='#003D7C', marker = "o", linestyle = 'None')
plt.plot(h2h_d_ante[4], regret_d_ante[4], color='#003D7C', marker = "o", linestyle = 'None')
plt.plot(h2h_d_ante[5], regret_d_ante[5], color='#003D7C', marker = "o", linestyle = 'None')
plt.plot(h2h_d_ante[6], regret_d_ante[6], color='#003D7C', marker = "o", linestyle = 'None')
plt.plot(h2h_d_ante[7], regret_d_ante[7], color='#003D7C', marker = "o", linestyle = 'None')
plt.plot(h2h_d_ante[8], regret_d_ante[8], color='#003D7C', marker = "o", linestyle = 'None')


plt.text(h2h_d_ante[0] + 0.01, regret_d_ante[0] - 0.005, 'd = '+str(deg_all[0]), color='#003D7C')
# plt.text(h2h_d[1] + 0.007, mci_d[1] - 0.003, 'd = 15', color='#003D7C')
plt.text(h2h_d_ante[1] + 0.01, regret_d_ante[1] - 0.005, 'd = '+str(deg_all[1]), color='#003D7C')
plt.text(h2h_d_ante[2] - 0.065, regret_d_ante[2] - 0.005, 'd = '+str(deg_all[2]), color='#003D7C')
plt.text(h2h_d_ante[4] + 0.01, regret_d_ante[4] - 0.005, 'd = '+str(deg_all[4]), color='#003D7C')
plt.text(h2h_d_ante[5] - 0.065, regret_d_ante[5] - 0.005, 'd = '+str(deg_all[5]), color='#003D7C')
plt.text(h2h_d_ante[6] - 0.065, regret_d_ante[6] - 0.005, 'd = '+str(deg_all[6]), color='#003D7C')
plt.text(h2h_d_ante[7] - 0.065, regret_d_ante[7] - 0.005, 'd = '+str(deg_all[7]), color='#003D7C')
plt.text(h2h_d_ante[8] - 0.065, regret_d_ante[8] - 0.005, 'd = '+str(deg_all[8]), color='#003D7C')


plt.plot(h2h_N_ante[0], regret_N_ante[0], color='#EF7C00', marker = "^", label = 'N',linestyle = 'None')
plt.text(h2h_N_ante[1] + 0.01, regret_N_ante[1] - 0.01, 'Baseline', color='red')
# plt.plot(h2h_N[1], mci_N[1], color='#EF7C00', marker = "^",linestyle = 'None')
plt.plot(h2h_N_ante[2], regret_N_ante[2], color='#EF7C00', marker = "^",linestyle = 'None')
plt.plot(h2h_N_ante[3], regret_N_ante[3], color='#EF7C00', marker = "^",linestyle = 'None')
plt.plot(h2h_N_ante[4], regret_N_ante[4], color='#EF7C00', marker = "^",linestyle = 'None')
plt.text(h2h_N_ante[0] + 0.01, regret_N_ante[0] - 0.01, 'N = 50', color='#EF7C00')
# plt.text(h2h_N[1] + 0.005, mci_N[1] - 0.003, 'N = 100', color='#EF7C00')
plt.text(h2h_N_ante[2] + 0.01, regret_N_ante[2] - 0.01, 'N = 200', color='#EF7C00')
plt.text(h2h_N_ante[3] + 0.01, regret_N_ante[3] - 0.01, 'N = 500', color='#EF7C00')
plt.text(h2h_N_ante[4] + 0.01, regret_N_ante[4] - 0.01, 'N = 1000', color='#EF7C00')


plt.plot(h2h_e_ante[0], regret_e_ante[0], color='grey', marker = "d", label = r'$\alpha$',linestyle = 'None')
# plt.plot(h2h_alpha[1], mci_alpha[1], color='grey', marker = "d",linestyle = 'None')
plt.plot(h2h_e_ante[2], regret_e_ante[2], color='grey', marker = "d",linestyle = 'None')
plt.plot(h2h_e_ante[3], regret_e_ante[3], color='grey', marker = "d",linestyle = 'None')
plt.text(h2h_e_ante[0] + 0.01, regret_e_ante[0] - 0.01, r'$\alpha$ = 0.25', color='grey')
# plt.text(h2h_alpha[1] + 0.005, mci_alpha[1] - 0.005, r'$\alpha$ = 1', color='grey')
plt.text(h2h_e_ante[2] - 0.075, regret_e_ante[2] - 0.01, r'$\alpha$ = 0.75', color='grey')
plt.text(h2h_e_ante[3] + 0.01, regret_e_ante[3] - 0.01, r'$\alpha$ = 1.0', color='grey')

plt.vlines(0.5, 0.0, 0.7, linestyle="dashed", alpha = 0.8,color = 'k')
plt.hlines(0.0, 0.4, 0.8, linestyle="dashed", alpha = 0.8,color = 'k')
plt.legend()

plt.xlabel('Head-to-head')
plt.ylabel('Regret reduction')

plt.plot(h2h_N_ante[1], regret_N_ante[1], color='red', marker = "o", linestyle = 'None')
# plt.annotate('Ridge vs. DDR', xy = (0.25,0.9), xycoords = 'axes fraction', bbox=dict(boxstyle="round", fc="w"), size = 10)

# fig.savefig(DataPath_parent+'DDR_vs_OLS_DPSN.eps', format='eps', bbox_inches="tight")
fig.savefig(Result_dir +'DDR_vs_OLS_diff_settings_ante.pdf', format='pdf', bbox_inches="tight")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize = (5, 5))
p_indices = [0,1,3,4,5]
plt.plot(h2h_P_ante[p_indices], regret_P_ante[p_indices], color='green', marker = "*", label = 'p', linestyle = 'None')
plt.text(h2h_P_ante[p_indices[0]] + 0.01, regret_P_ante[p_indices[0]] - 0.005, 'p='+str(num_feat_all[p_indices[0]]), color='green')
plt.text(h2h_P_ante[p_indices[1]] + 0.01, regret_P_ante[p_indices[1]] - 0.005, 'p='+str(num_feat_all[p_indices[1]]), color='green')
plt.text(h2h_P_ante[p_indices[2]] - 0.05, regret_P_ante[p_indices[2]] - 0.005, 'p='+str(num_feat_all[p_indices[2]]), color='green')
plt.text(h2h_P_ante[p_indices[3]] - 0.05, regret_P_ante[p_indices[3]] - 0.005, 'p='+str(num_feat_all[p_indices[3]]), color='green')
plt.text(h2h_P_ante[p_indices[4]] - 0.065, regret_P_ante[p_indices[4]] - 0.005, 'p='+str(num_feat_all[p_indices[4]]), color='green')

plt.plot(h2h_d_ante[0], regret_d_ante[0], color='#003D7C', marker = "o", label = 'd', linestyle = 'None')
# plt.plot(h2h_d[1], mci_d[1], color='#003D7C', marker = "o", linestyle = 'None')
plt.plot(h2h_d_ante[1], regret_d_ante[1], color='#003D7C', marker = "o", linestyle = 'None')
plt.plot(h2h_d_ante[2], regret_d_ante[2], color='#003D7C', marker = "o", linestyle = 'None')
plt.plot(h2h_d_ante[4], regret_d_ante[4], color='#003D7C', marker = "o", linestyle = 'None')
plt.plot(h2h_d_ante[5], regret_d_ante[5], color='#003D7C', marker = "o", linestyle = 'None')
plt.plot(h2h_d_ante[6], regret_d_ante[6], color='#003D7C', marker = "o", linestyle = 'None')
plt.plot(h2h_d_ante[7], regret_d_ante[7], color='#003D7C', marker = "o", linestyle = 'None')
plt.plot(h2h_d_ante[8], regret_d_ante[8], color='#003D7C', marker = "o", linestyle = 'None')


plt.text(h2h_d_ante[0] + 0.01, regret_d_ante[0] - 0.005, 'd = '+str(deg_all[0]), color='#003D7C')
# plt.text(h2h_d[1] + 0.007, mci_d[1] - 0.003, 'd = 15', color='#003D7C')
plt.text(h2h_d_ante[1] + 0.01, regret_d_ante[1] - 0.005, 'd = '+str(deg_all[1]), color='#003D7C')
plt.text(h2h_d_ante[2] - 0.065, regret_d_ante[2] - 0.005, 'd = '+str(deg_all[2]), color='#003D7C')
plt.text(h2h_d_ante[4] + 0.01, regret_d_ante[4] - 0.005, 'd = '+str(deg_all[4]), color='#003D7C')
plt.text(h2h_d_ante[5] - 0.065, regret_d_ante[5] - 0.005, 'd = '+str(deg_all[5]), color='#003D7C')
plt.text(h2h_d_ante[6] - 0.065, regret_d_ante[6] - 0.005, 'd = '+str(deg_all[6]), color='#003D7C')
plt.text(h2h_d_ante[7] - 0.065, regret_d_ante[7] - 0.005, 'd = '+str(deg_all[7]), color='#003D7C')
plt.text(h2h_d_ante[8] - 0.065, regret_d_ante[8] - 0.005, 'd = '+str(deg_all[8]), color='#003D7C')


plt.plot(h2h_N_ante[0], regret_N_ante[0], color='#EF7C00', marker = "^", label = 'N',linestyle = 'None')
plt.text(h2h_N_ante[1] + 0.01, regret_N_ante[1] - 0.01, 'Baseline', color='red')
# plt.plot(h2h_N[1], mci_N[1], color='#EF7C00', marker = "^",linestyle = 'None')
plt.plot(h2h_N_ante[2], regret_N_ante[2], color='#EF7C00', marker = "^",linestyle = 'None')
plt.plot(h2h_N_ante[3], regret_N_ante[3], color='#EF7C00', marker = "^",linestyle = 'None')
plt.plot(h2h_N_ante[4], regret_N_ante[4], color='#EF7C00', marker = "^",linestyle = 'None')
plt.text(h2h_N_ante[0] + 0.01, regret_N_ante[0] - 0.01, 'N = 50', color='#EF7C00')
# plt.text(h2h_N[1] + 0.005, mci_N[1] - 0.003, 'N = 100', color='#EF7C00')
plt.text(h2h_N_ante[2] + 0.01, regret_N_ante[2] - 0.01, 'N = 200', color='#EF7C00')
plt.text(h2h_N_ante[3] + 0.01, regret_N_ante[3] - 0.01, 'N = 500', color='#EF7C00')
plt.text(h2h_N_ante[4] + 0.01, regret_N_ante[4] - 0.01, 'N = 1000', color='#EF7C00')


plt.plot(h2h_e_ante[0], regret_e_ante[0], color='grey', marker = "d", label = r'$\alpha$',linestyle = 'None')
# plt.plot(h2h_alpha[1], mci_alpha[1], color='grey', marker = "d",linestyle = 'None')
plt.plot(h2h_e_ante[2], regret_e_ante[2], color='grey', marker = "d",linestyle = 'None')
plt.plot(h2h_e_ante[3], regret_e_ante[3], color='grey', marker = "d",linestyle = 'None')
plt.text(h2h_e_ante[0] + 0.01, regret_e_ante[0] - 0.01, r'$\alpha$ = 0.25', color='grey')
# plt.text(h2h_alpha[1] + 0.005, mci_alpha[1] - 0.005, r'$\alpha$ = 1', color='grey')
plt.text(h2h_e_ante[2] - 0.075, regret_e_ante[2] - 0.01, r'$\alpha$ = 0.75', color='grey')
plt.text(h2h_e_ante[3] + 0.01, regret_e_ante[3] - 0.01, r'$\alpha$ = 1.0', color='grey')

plt.vlines(0.5, 0.0, 0.7, linestyle="dashed", alpha = 0.8,color = 'k')
plt.hlines(0.0, 0.4, 0.8, linestyle="dashed", alpha = 0.8,color = 'k')
plt.legend()

plt.xlabel('Head-to-head')
plt.ylabel('Regret reduction')

plt.plot(h2h_N_ante[1], regret_N_ante[1], color='red', marker = "o", linestyle = 'None')
# plt.annotate('Ridge vs. DDR', xy = (0.25,0.9), xycoords = 'axes fraction', bbox=dict(boxstyle="round", fc="w"), size = 10)

# fig.savefig(DataPath_parent+'DDR_vs_OLS_DPSN.eps', format='eps', bbox_inches="tight")
fig.savefig(Result_dir +f'DDR_vs_OLS_diff_settings_ante_mu={mu}_lamb={lamb}.pdf', format='pdf', bbox_inches="tight")
plt.show()